In [1]:
from pdf2image import convert_from_path
import os
import pytesseract
from pytesseract import Output
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

POPPLER_PATH = os.getenv("POPPLER_PATH")
pytesseract.pytesseract.tesseract_cmd = os.getenv("PYTESSERACT_PATH")

In [2]:
class Token:
    def __init__(self, type, value, bbox):
        self.type = type   # e.g., "FIELD_LABEL"
        self.value = value # e.g., "Name:"
        self.bbox = bbox   # (x, y, w, h)

In [15]:
pages = convert_from_path(
    "./forms/Application-Form-School.pdf", 
    300,
    poppler_path=POPPLER_PATH
)

data = pytesseract.image_to_data(pages[0], output_type=Output.DICT)

In [16]:
# Print the first few words detected to prove it works
# print("First 5 words found:", data['width'][:20])
print("Detected text data keys:", data.keys())
df = pd.DataFrame(data)
df.head(20)

Detected text data keys: dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,2550,3300,-1,
1,2,1,1,0,0,0,412,258,1716,135,-1,
2,3,1,1,1,0,0,412,258,1716,135,-1,
3,4,1,1,1,1,0,412,258,1716,135,-1,
4,5,1,1,1,1,1,412,259,784,134,96,Application
5,5,1,1,1,1,2,1257,259,344,107,96,Form
6,5,1,1,1,1,3,1656,258,472,108,96,School
7,2,1,2,0,0,0,306,424,1950,17,-1,
8,3,1,2,1,0,0,306,424,1950,17,-1,
9,4,1,2,1,1,0,306,424,1950,17,-1,


In [17]:
tokens = []
print("First 5 words found:", data['text'][:150])


First 5 words found: ['', '', '', '', 'Application', 'Form', 'School', '', '', '', ' ', '', '', '', 'Student', 'Information', '', '', '', 'e', 'Student', 'Name:', '', 'e', 'Age:', '', 'e', 'Current', 'School:', '', '', '', 'e', 'Grade:', '', '', '', 'Parent/Guardian', 'Details', '', '', '', 'e', 'Parent/Guardian', 'Name:', '', 'e', 'Contact', 'Number:', '', '', '', 'e', 'Email', 'Address:', '', '', '', 'Previous', 'Academic', 'Record', '', '', '', 'Subject', 'Grades', '', '', '', 'Extra-Curricular', 'Activities', '', '', '', "Teacher's", 'Comments', '', '', '', 'Copyright', '@', 'SampleForms.com']
